**Read in and pre-process data**

In [ ]:
# Download data
download.file("http://crr.ugent.be/blp/txt/blp-items.txt.zip", "blp.zip") 
# Open connection to .txt file in .zip archive
conn = unz("blp.zip","blp-items.txt") 
# Read data
blp = read.table(conn, head=TRUE)
# Remove words without RT
blp = blp[-which(is.na(blp$rt)),]

Inspect data

In [ ]:
# number of rows(words) and columns
dim(blp)

In [ ]:
# average response time
mean(blp$rt)

In [ ]:
# the average response time for the word "hear"
blp$rt[which(blp$spelling == 'hear')]
# "dear"
blp$rt[which(blp$spelling == 'dear')]

In [ ]:
# the word with the longest average response time
blp$spelling[which(blp$rt == max(blp$rt))]

In [ ]:
# Plot the distribution of the response times in the BLP. 
plot(density(blp$rt), xlab = 'response time(ms)', ylab = 'density')

Add 'status' colunm

In [ ]:
# Add a column to the data that contains the event status for all words.
blp$status = 1
# Create temporary data frame
blp_tmp = blp
# Update response times
blp_tmp$rt[which(blp_tmp$rt>1000)] = 1000
blp_tmp$status[which(blp_tmp$rt == 1000)] = 0
# inspect
blp_tmp$status[which(blp_tmp$rt == 1000)]

In [ ]:
# how many words have an event status of 0
length(blp_tmp$status[which(blp_tmp$rt == 1000)])

In [ ]:
# What percentage of the data do these words represent?
length(blp_tmp$status[which(blp_tmp$rt == 1000)])/length(blp$rt)

**Objective functions**

In [ ]:
## PDF

# Generate the probability density function for the response times in the BLP
d = density(blp$rt, from = 300, to = 1650, n = 1351)
# What is the probability density estimate for t = 600? 
d$y[which(d$x == 600)]
# What is the probability density estimate for t = 900? 
d$y[which(d$x == 900)]
# What is the relative likelihood of a response time of 600 ms as compared to a response time of 900 ms?
d$y[which(d$x == 600)]/d$y[which(d$x == 900)]
# At what point in time does the probability density function reach its maximum?
d$x[which(d$y==max(d$y))]
# Plot the probability density function for the response times in the BLP. 
plot(d)
# How does the probability density function for the lexical decision latencies in the BLP compare to the probability density function for the naming latencies in the ELP?
head(blp)

In [ ]:
## CDF

# Generate the cumulative density function for the response times in the BLP.
cdf = ecdf(blp$rt)

# Access the environment of cdf
cdf = environment(cdf)

# What proportion of words has been responded to at 600ms after stimulus onset? 
cdf$y[which(cdf$x==600)]
# How about at 900 ms after stimulus onset?
cdf$y[which(cdf$x==900)]
# Plot the cumulative density function for the response times in the BLP.
plot(ecdf(blp$rt), xlab = "response time (ms)", ylab = "F(x)", main = "",
     xlim = c(0,3000))

In [ ]:
## SURVIVAL


# Load survival library
library(survival)

# All words in the BLP were responded to. Create a column status that is set to 1 for all words.
blp$status = 1

# Create a survival object that contains the time event tuple for all words.
surv = Surv(time = blp$rt, event = blp$status)

# Round the response latencies(rt) in the BLP to the nearest integer using the round() function. 
blp$rt = round(blp$rt)

# Next, generate the Kaplan-Meier estimate of the survival function for the lexical decision latencies(rt) in the BLP.
km_estimator = survfit(surv~1)

# Plot the estimated survival curve.
plot(km_estimator, xlab = "rounded rt (ms)", ylab = "S(t)", conf.int = FALSE)

# What proportion of words has not been responded to at 600 ms after stimulus onset? 
km_estimator$surv[which(km_estimator$time == 600)]
# How about at 900 ms after stimulus onset? 
km_estimator$surv[which(km_estimator$time == 900)]
# What is the relation between the survival function and the cumulative density function?

# a simple illustration
hold = TRUE

for (rt in blp$rt)
    {
    if (km_estimator$surv[which(km_estimator$time == rt)] + cdf$y[which(cdf$x==rt)] != 1)
        {print('S(t) = 1 − F(t) does not hold???because of rounded???')
         hold = FALSE
        break}
}

if (hold){print('S(t) = 1 − F(t)')}


Generate the hazard function for the response times in the BLP

In [ ]:
## Hazard


# Restrict density function time points
d$y = d$y[which(d$x%in%km_estimator$time)]
d$x = d$x[which(d$x%in%km_estimator$time)]
# Define hazard function
hazard = d$y / km_estimator$surv

# Hazard plot
plot(km_estimator$time, hazard, ylim = c(0, 0.1), xlab = "rounded response time (ms)",
     ylab = expression(lambda(t)),type="l")

# Cumulative hazard function plot
plot(km_estimator, cumhaz = TRUE, conf.int = FALSE, xlab = "rounded response time (ms)",
     ylab = expression(Lambda(t)))

**Categorical predictors**

In [ ]:
# Load survival library
library(survival)

# Round response times to integers
blp$rt = round(blp$rt)

# How many stimuli are present in the data?
nrow(blp)

# Visualize the number of words and nonwords in the data with a barplot.
tab = table(blp$lexicality)
tab

options(repr.plot.width=5, repr.plot.height=5)
barplot(tab, xlab = "lexicality", ylab = "number of stimuli", ylim = c(0, 30000))

# Add a column with the event status to the data frame. Set the event status to 1 for each trial. 
blp$status = 1

# Next, create a survival object with the time-event tuple for the lexical decision data.
surv = Surv(time = blp$rt, event = blp$status)

# Fit survival curves to the data with the survfit() function. 
km_estimator = survfit(surv~1)

# What are the median response times for words and for nonwords?
median(blp$rt[which(blp$lexical == 'W')])
median(blp$rt[which(blp$lexical == 'N')])

# alternative solution

survival = survfit(surv ~ lexicality, data = blp)
survival

# Download survival package
devtools::install_github("kassambara/survminer", build_vignettes = FALSE, upgrade = "never", lib="/kaggle/working")


# Load survival library
library(survminer)

# Plot the probability density function for words
plot(density(blp$rt[which(blp$lexicality == "W")]), xlab = "time (ms)",
     ylab = "pdf", main = "", ylim = c(0, 0.008))
# Add probability density function for nonwords
lines(density(blp$rt[which(blp$lexicality == "N")]), col = "blue")

# survival plot
options(repr.plot.width=10, repr.plot.height=5)
plot(survival, xlab = "time (ms)", ylab = "S(t)", col = c("black", "blue"))

# log-rank test
surv.test = survdiff(surv ~ lexicality, data = blp)
surv.test




In [ ]:
# Download data
download.file("http://crr.ugent.be/blp/txt/blp-stimuli.txt.zip",
              "blp.stimuli.zip")
# Open connection to .txt file in .zip archive
conn = unz("blp.stimuli.zip","blp-stimuli.txt")
# Read data
blp.stimuli = read.table(conn, head=TRUE, sep="\t")

# in spect data
head(blp.stimuli)


In [ ]:
# Restrict to the relevant columns
blp.stimuli = blp.stimuli[,c("spelling", "synclass")]
# Restrict to relevant parts-of-speech categories
blp.stimuli = blp.stimuli[which(blp.stimuli$synclass%in%
  c("Noun", "Verb", "Adjective", "Adverb")),]
# Drop unused levels from the data frame
blp.stimuli = droplevels(blp.stimuli)

# inspect restricted data
head(blp.stimuli)

A barplot to visualize the distribution of parts-of-speech categories.

In [ ]:

tab = table(blp.stimuli$synclass)
# bar-plot
barplot(tab, xlab = "synclass", ylab = "number of trials")

In [ ]:
# add the lexical information to the data frame with the join() function from the plyr package.

# Download survival package
install.packages("plyr")
# Load plyr library
library(plyr)

# use the join() function to add the lexical information to the data frame with the response times

# Add the lexical information
blp = join(blp, blp.stimuli, type = "inner")

head(blp)

In [ ]:
# Fit survival curves for the parts-of-speech categories. Inspect the results.

# Load library
library(survival)

# Generate survival curves
surv.cat.pos = survfit(Surv(rt, status) ~ synclass, data = blp)


surv.cat.pos


The value of the survival function for each parts-of-speech category at 600 ms after stimulus onset

In [ ]:
# Load survminer library
library(survminer)
# Create summary of survival curve
surv.cat.pos.sum = surv_summary(surv.cat.pos)

# Define columns of interest
cols = c("time","surv","upper","lower","synclass")
# Inspect survival curves at 600 ms
surv.cat.pos.sum[which(surv.cat.pos.sum$time==600), cols]
# Also retrieve the probability of survival for each parts-of-speech category at 900 ms after stimulus onset. 
surv.cat.pos.sum[which(surv.cat.pos.sum$time==900), cols]

In [ ]:
# Plot the survival curves for the parts-of-speech categories.
plot(surv.cat.pos, xlab = "time (ms)", ylab = "S(t)",
     col = c("#00a37a", "#000aa3", "#a30000", "#a300a3"))

In [ ]:
# Statistical test: log-rank test
surv.cat.pos.test = survdiff(Surv(rt, status) ~ synclass,
  data = blp)

# Inspect
surv.cat.pos.test

conlude from the log-rank test inspection: <br>
The lower the p−value, the more confidently we can reject the null hypothesis
p-value is significatly small, we can reject the null hypothesis



In [ ]:
# Statistical test: post-hoc test for pairwise differences
surv.cat.pos.pairwise.test = pairwise_survdiff(Surv(rt, status) ~ synclass,
  data = blp, p.adjust.method = "none")

# Inspect
surv.cat.pos.pairwise.test